In [2]:
import os
from dotenv import load_dotenv
load_dotenv('azure.env')

True

In [95]:
from langchain_openai import AzureChatOpenAI, ChatOpenAI
llm = AzureChatOpenAI(api_key = os.environ["AZURE_OPENAI_API_KEY"],  
                      api_version="2024-07-01-preview",
                      azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
                      azure_deployment= "gpt-4o",
                      temperature = 1,
                      model="gpt-4o",
                      streaming=True)

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from langchain_community.tools.bing_search import BingSearchResults
from langchain_community.utilities import BingSearchAPIWrapper
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

api_wrapper = BingSearchAPIWrapper()
tools = [BingSearchResults(api_wrapper = api_wrapper,num_results=10)]

In [ ]:
async_browser = create_async_playwright_browser(headless=True)
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = tools + toolkit.get_tools()

In [74]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.prompts.chat import (
    BaseMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    PromptTemplate,
    ImagePromptTemplate
)

In [150]:
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.agents.output_parsers.openai_functions import OpenAIFunctionsAgentOutputParser
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.runnables import RunnableConfig
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.messages import AIMessageChunk
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages


system_prompt = f"""You are an expert ads creator. You are tasked to add the Sponsored Content ads in between a given blog or website. 
\\ You will be provided a link to the blog. If you are tasked to create the ads about addidas in a personal blog website. 
\\ You first understand the blog and build a narative about the ad you will put in a such a way that it will look natural for the reader. 
\\ Only return the sponsored ad content note more than 2 paragraphs
\\ Be Creative in your ad , make it more contextual. 
\\ Add Images if provided and return html div"""

system_prompt = """You are an expert ads creator specializing in native advertising. Your task is to seamlessly integrate sponsored content ads into existing blog posts or website.

** Instructions:

 - Analyze the provided blog post or website content thoroughly.
 - Navigate to sponsored ad Link to get more info about the sponsored ad
 - Craft a narrative for the ad that feels like a natural extension of the blog's content.
 - Return the sponsored content in an HTML div format.

** Guidelines for ad creation:

 - Match the tone and style of the blog
 - Ensure the ad is contextually relevant and creative, blending seamlessly with the surrounding content.
 - Limit the ad to a maximum of two paragraphs.
 - If images are provided, incorporate them appropriately.
 - Use language and themes that resonate with the blog's audience
 - Be subtle in your approach – the ad should not feel intrusive or overly promotional

Output format:
<div class="sponsored-content">
    [Your two-paragraph ad here]
    [Include image HTML if provided]
</div>
Remember to be creative, contextual, and natural in your ad creation to maintain the reader's engagement while effectively promoting the promption link"""

text_prompt = ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], 
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_prompt)), 
                                      MessagesPlaceholder(variable_name='chat_history', optional=True), 
                                      HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), 
                                      MessagesPlaceholder(variable_name='agent_scratchpad')])


image_prompt = ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'],   
                                  messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_prompt)), 
                                      MessagesPlaceholder(variable_name='chat_history', optional=True), 
                                      HumanMessagePromptTemplate(prompt=[PromptTemplate(input_variables=['input'], template='{input}'),
                                                                        ImagePromptTemplate(input_variables=['img'], template={'url': '{img}'})]),
                                      MessagesPlaceholder(variable_name='agent_scratchpad')])

llm_with_tools = llm.bind_tools(
    tools=tools
)

def prompt_router(input):
    if input["img"]:
        return image_prompt
    return text_prompt

def print_x(x):
    print(x)

agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        )
    )
    | RunnableLambda(prompt_router)
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [151]:
import base64
import magic
mime = magic.Magic(mime=True)

image_path = 'data/chandu_champion.png'
with open(image_path, "rb") as image_file:
    image_base64 = base64.b64encode(image_file.read()).decode('utf-8')

In [152]:
prompt="""Blog Link: https://trends.glance.com/newz/space/IN/en/feedpost-now/f-cc428ee2a0-cf888ef9-bafb-5676-b323-86be059fc469
sponsored ad: https://www.primevideo.com/detail/Chandu-Champion/0MARCAWG5VJK4GIDNLXR1CR0ON
"""
input_dict = {"input": prompt, "img": f"data:{mime.from_file(image_path)};base64,{image_base64}", "chat_history": []}
image_link = "https://static.tnn.in/thumb/msid-110986320,width-400,height-200,resizemode-75/110986320.jpg?quality=100"
input_dict = {"input": prompt, "img": None, "chat_history": []}

In [155]:
config = RunnableConfig(callbacks = [StreamingStdOutCallbackHandler()])
async for patch in agent_executor.astream_log(input_dict, config = config):
    for op in patch.ops:
        if op["op"] != "add":
            continue
        value = op["value"]
        if not isinstance(value, AIMessageChunk):
            continue
        if value.content == "":
            continue
        #await msg.stream_token(value.content)

```html
<div class="sponsored-content">
    While exploring the wonders of space through cinema, let's not forget the incredible tales of human spirit and determination that inspire us on Earth as well. One such story is "Chandu Champion," a gripping film about a man who faced numerous challenges with unwavering zeal, ultimately winning India's first individual gold medal in the Olympics. Just like your favorite space epics, this movie offers breathtaking adventures and a profound narrative, making it a must-watch for anyone who loves inspirational stories.
    Don't miss out on "Chandu Champion" on Prime Video, where you can witness this epic journey of perseverance and triumph. The film promises to captivate you with its historical drama and compelling storytelling, echoing the same sense of awe and inspiration that space movies bring to the big screen.
</div>
```

In [154]:
from IPython.display import display, HTML
display(HTML(value.content))